In [1]:
import os
import nltk
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from groq import Groq
import time
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Define workspace path
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"

# Set environment variables to store large files in workspace
os.environ["HF_HOME"] = os.path.join(WORKSPACE_PATH, "huggingface")
os.environ["TRANSFORMERS_CACHE"] = os.path.join(WORKSPACE_PATH, "huggingface")
os.environ["HF_DATASETS_CACHE"] = os.path.join(WORKSPACE_PATH, "datasets")
os.environ["NLTK_DATA"] = os.path.join(WORKSPACE_PATH, "nltk_data")

# Ensure necessary directories exist
os.makedirs(os.environ["HF_HOME"], exist_ok=True)
os.makedirs(os.environ["TRANSFORMERS_CACHE"], exist_ok=True)
os.makedirs(os.environ["HF_DATASETS_CACHE"], exist_ok=True)
os.makedirs(os.environ["NLTK_DATA"], exist_ok=True)
BM25_INDEX_PATH = os.path.join(WORKSPACE_PATH, "bm25_index.pkl")
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
NLTK_PATH = os.path.join(WORKSPACE_PATH, "nltk_data")

# Ensure directory exists
os.makedirs(NLTK_PATH, exist_ok=True)

# Set NLTK data path
nltk.data.path.append(NLTK_PATH)

# Force download punkt & stopwords
nltk.download("punkt", download_dir=NLTK_PATH, force=True)
nltk.download("stopwords", download_dir=NLTK_PATH, force=True)
nltk.download("wordnet", download_dir=NLTK_PATH, force=True)
print(f"Workspace set up at: {WORKSPACE_PATH}")

[nltk_data] Downloading package punkt to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...


Workspace set up at: /data/horse/ws/uskh580e-myws


In [2]:
from datasets import load_dataset

# Load the ELI5 dataset from sentence-transformers
eli5 = load_dataset("sentence-transformers/eli5", split="train")



In [3]:
eli5_df = eli5.to_pandas()
print(eli5_df.columns)
eli5_df.head(3)


Index(['question', 'answer'], dtype='object')


,question,answer
0,in football whats the point of wasting the fir...,"Keep the defense honest, get a feel for the pa..."
1,Why are different tiers (regular < mid < premi...,"As someone who uses quality Premium, I wish th..."
2,Stars and Visibility,It's a quirk of the human eye. At the center o...


In [4]:
import re

def normalize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)    # strip punctuation
    text = re.sub(r"\s+", " ", text).strip()   # collapse whitespace
    return text

# apply normalization
eli5_df["question_norm"] = eli5_df["question"].apply(normalize)
eli5_df["answer_norm"]   = eli5_df["answer"].apply(normalize)


In [5]:
eli5_df[["question", "question_norm", "answer", "answer_norm"]].head(5)


,question,question_norm,answer,answer_norm
0,in football whats the point of wasting the fir...,in football whats the point of wasting the fir...,"Keep the defense honest, get a feel for the pa...",keep the defense honest get a feel for the pas...
1,Why are different tiers (regular < mid < premi...,why are different tiers regular mid premium of...,"As someone who uses quality Premium, I wish th...",as someone who uses quality premium i wish thi...
2,Stars and Visibility,stars and visibility,It's a quirk of the human eye. At the center o...,its a quirk of the human eye at the center of ...
3,How do we know all the money the government is...,how do we know all the money the government is...,I'm pretty confident most of it isn't going ba...,im pretty confident most of it isnt going back...
4,What are good and bad sides of manual and auto...,what are good and bad sides of manual and auto...,"Automatics weigh more, so that alone makes gas...",automatics weigh more so that alone makes gas ...


In [6]:
out_path = os.path.join(WORKSPACE_PATH, "eli5_normed_slice1k.jsonl")
eli5_df.to_json(out_path, orient="records", lines=True)
print("✅ Written normalized ELI5 to", out_path)


✅ Written normalized ELI5 to /data/horse/ws/uskh580e-myws/eli5_normed_slice1k.jsonl


In [7]:
import os
print(os.listdir("/data/horse/ws/uskh580e-llm_citation_ws"))


['.ipynb_checkpoints', 'prompting_cot.ipynb', 'myenv', 'retrieval_indices', 'new_env', 'postgen.ipynb', 'litsearch_llm_rag.ipynb', 'ELI5.ipynb', 'non_rag_baseline_all_strategies.csv', 'prompting.ipynb', 'litsearch.ipynb', 'ALCE.ipynb', 'datasets.ipynb', 'ALCE', 'eli5_normed_slice1k.jsonl', 'BetterRetrieval.ipynb', 'myenv2', 'prompting_cot_scads.ipynb']


In [8]:
import os, json, pickle
import numpy as np
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# 1. Paths
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
ELI5_JSONL    = os.path.join(WORKSPACE_PATH, "eli5_normed_slice1k.jsonl")
BM25_INDEX    = os.path.join(WORKSPACE_PATH, "eli5_bm25_index.pkl")
CORPUS_TXT    = os.path.join(WORKSPACE_PATH, "eli5_corpus_texts.pkl")

# 2. Load normalized ELI5 as list of dicts
docs = []
with open(ELI5_JSONL, "r") as f:
    for line in f:
        row = json.loads(line)
        docs.append({
            "question": row["question_norm"],
            "answer"  : row["answer_norm"]
        })
print(f"Loaded {len(docs)} QA pairs from normalized ELI5.")

# 3. Build tokenized corpus from the answers
stop_words = set(stopwords.words("english"))
tokenized_corpus = []
for entry in docs:
    tokens = word_tokenize(entry["answer"])
    tokenized_corpus.append([t for t in tokens if t not in stop_words])

# 4. Create and save BM25 index
bm25 = BM25Okapi(tokenized_corpus)
with open(BM25_INDEX, "wb") as f:
    pickle.dump(bm25, f)
# Also save raw texts for lookup
with open(CORPUS_TXT, "wb") as f:
    pickle.dump([e["answer"] for e in docs], f)

print("✅ BM25 index and corpus texts saved.")

# 5. Example retrieval function
def retrieve_top_k(question, k=5):
    q_tokens = [t for t in word_tokenize(question) if t not in stop_words]
    scores = bm25.get_scores(q_tokens)
    top_idxs = np.argsort(scores)[-k:][::-1]
    return [(i, docs[i]["answer"], float(scores[i])) for i in top_idxs]

# 6. Sanity check
sample_q = docs[0]["question"]
print("Sample norm question:", sample_q)
for idx, ans, sc in retrieve_top_k(sample_q, k=3):
    print(f"\n→ Doc #{idx} (score {sc:.2f}):\n{ans}")


Loaded 325475 QA pairs from normalized ELI5.
✅ BM25 index and corpus texts saved.
Sample norm question: in football whats the point of wasting the first two plays with a rush up the middle not regular rush plays i get those

→ Doc #83666 (score 30.16):
with football theyre the best in the world at american football because nobody else plays it

→ Doc #5120 (score 29.06):
i actually get a huge rush from solving a math problem you get the dopamine rush from doing things you like partially if you dont like something you wont get the rush something like flappy bird is easy for most people to enjoy so you get the rush

→ Doc #31767 (score 26.55):
its more an adrenaline addiction people get a rush when they take a perceived risk do it more it takes more adrenaline to get the same rush you are messing with the chemical balance of the brain


In [9]:
import requests
# ─────────────────────────────────────────────────────────────────────────────
# 1) Scads.ai settings
# ─────────────────────────────────────────────────────────────────────────────
SCADS_API_KEY = os.getenv("SCADS_API_KEY", "sk-D69vfwcl4A05QbVhV5yxyA")  # or hardcode if you must
SCADS_URL     = "https://llm.scads.ai/v1/chat/completions"
MODEL_NAME    = "meta-llama/Llama-3.3-70B-Instruct"
RETRIEVE_K    = 5


# ─────────────────────────────────────────────────────────────────────────────
# 2) New helper: retrieve + prompt via Scads.ai
# ─────────────────────────────────────────────────────────────────────────────
def answer_with_citations_scads(question: str,
                                k: int = RETRIEVE_K,
                                strategy: str = None) -> str:
    """
    1) retrieves top-k docs for `question`
    2) prompts Scads.ai (single- or two-stage) to answer with citations
    """
    # ——— a) retrieve your passages —————————————
    hits = retrieve_top_k(question, k=k)            # e.g. [(doc_id, text, score), …]

    # ——— b) build the context blob ————————————
    context = "\n\n".join(
        f"Corpus ID: {doc_id}\nFull Text: {text[:1000]}…"  # truncate for safety
        for doc_id, text, _ in hits
    )

    # ——— c) two‑stage “Academic Aspiration 2‑stage” ——————
    if strategy == "Academic Aspiration (2-stage)":
        # Stage 1: send your role‑setting prompt
        role_payload = {
            "model": MODEL_NAME,
            "messages": [
                {"role": "user", "content": question}
            ]
        }
        role_resp = requests.post(
            SCADS_URL,
            headers={
                "Authorization": f"Bearer {SCADS_API_KEY}",
                "Content-Type":    "application/json"
            },
            json=role_payload
        )
        role_resp.raise_for_status()
        assistant_ack = role_resp.json()["choices"][0]["message"]["content"]

        # Stage 2: send the real Q + context
        prompt = f"""{assistant_ack}

Only cite sources that are directly relevant and verifiable from the retrieved documents below.  
Use this format for each citation: [Corpus ID].

Retrieved Documents:
{context}

Question: {question}
Answer:"""

    else:
        # single‑stage prompt
        prompt = f"""
You are a helpful assistant.  Cite every factual claim you make using [Corpus ID] from the docs below.

Retrieved Documents:
{context}

Question: {question}
Answer:"""

    # ——— d) final call ————————————————
    final_payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.0,
        "max_tokens": 512,
    }
    resp = requests.post(
        SCADS_URL,
        headers={
            "Authorization": f"Bearer {SCADS_API_KEY}",
            "Content-Type":    "application/json"
        },
        json=final_payload
    )
    resp.raise_for_status()
    return resp.json()["choices"][0]["message"]["content"].strip()


# ─────────────────────────────────────────────────────────────────────────────
# 3) Quick sanity check
# ─────────────────────────────────────────────────────────────────────────────
q0 = "in football whats the point of wasting the first two plays with a rush up the middle?"
print(answer_with_citations_scads(q0, strategy="Academic Aspiration (2-stage)"))


The point of starting a drive with two consecutive running plays up the middle in football is to establish the run, set up play-action, wear down the defense, get a feel for the defense, play it safe, create a rhythm, and neutralize the pass rush. As stated, football coaches are highly involved in making decisions about plays and strategies [Corpus ID: 156076]. This approach can help the offense create opportunities for bigger plays later on.


In [26]:
import os
print(os.listdir("/data/horse/ws/uskh580e-llm_citation_ws"))


['.ipynb_checkpoints', 'prompting_cot.ipynb', 'myenv', 'retrieval_indices', 'new_env', 'postgen.ipynb', 'litsearch_llm_rag.ipynb', 'ELI5.ipynb', 'non_rag_baseline_all_strategies.csv', 'prompting.ipynb', 'litsearch.ipynb', 'ALCE.ipynb', 'datasets.ipynb', 'ALCE', 'eli5_normed_slice1k.jsonl', 'BetterRetrieval.ipynb', 'myenv2', 'prompting_cot_scads.ipynb']


In [11]:
import os
import re
import json
import pandas as pd
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ─────────────────────────────────────────────────────────────────────────────
# 1) Read your data
# ─────────────────────────────────────────────────────────────────────────────
WORKSPACE_PATH = "/data/horse/ws/uskh580e-llm_citation_ws"
ELI5_PATH      = os.path.join(WORKSPACE_PATH, "eli5_normed_slice1k.jsonl")
records        = [json.loads(line) for line in open(ELI5_PATH, "r")]
print(f"Loaded {len(records)} examples")

# ─────────────────────────────────────────────────────────────────────────────
# 2) Load NLI model/tokenizer onto CPU (float32)
# ─────────────────────────────────────────────────────────────────────────────
device = "cpu"
nli_model_name = "facebook/bart-large-mnli"

nli_tokenizer = AutoTokenizer.from_pretrained(nli_model_name)
nli_model     = AutoModelForSequenceClassification.from_pretrained(
    nli_model_name,
    low_cpu_mem_usage=True
).to(device)     # default dtype=float32 on CPU
nli_model.eval()

# ─────────────────────────────────────────────────────────────────────────────
# 3) Batched entailment checker
# ─────────────────────────────────────────────────────────────────────────────
def batch_entails(pairs, batch_size=16):
    """
    pairs: List of (premise: str, hypothesis: str)
    returns: List[bool] of same length: True if entailment label==2
    """
    results = []
    for i in range(0, len(pairs), batch_size):
        chunk = pairs[i : i + batch_size]
        ps, hs = zip(*chunk)
        toks = nli_tokenizer(
            list(ps),
            list(hs),
            return_tensors="pt",
            truncation=True,
            padding=True
        ).to(device)
        with torch.no_grad():
            logits = nli_model(**toks).logits
        preds = torch.argmax(logits, dim=-1).cpu().tolist()
        results.extend([p == 2 for p in preds])
    return results

# ─────────────────────────────────────────────────────────────────────────────
# 4) Citation‐evaluation loop
# ─────────────────────────────────────────────────────────────────────────────
cite_pattern = re.compile(r"\[Corpus ID:\s*(\d+)\]")
RETRIEVE_K   = 20

out_rows = []
for ex in tqdm(records, desc="Scoring examples", total=len(records)):
    q = ex["question"]

    # a) get your model’s answer with “[Corpus ID: X]”
    ans = answer_with_citations_scads(q, strategy="Academic Aspiration (2-stage)")

    # b) extract all cited IDs
    cited_ids = [int(m) for m in cite_pattern.findall(ans)]

    # c) re-retrieve top‑K for this query
    hits = retrieve_top_k(q, k=RETRIEVE_K)
    # hits is a list of (doc_id, doc_text, score)
    retrieved_ids = [doc_id for doc_id, _, _ in hits]

    #────────────────────────────────────────
    # build all (premise, hypothesis) checks
    #────────────────────────────────────────
    pairs = []
    # 1) joint concat check
    concat_cited = "\n\n".join(text for doc_id, text, _ in hits if doc_id in cited_ids)
    if cited_ids:
        pairs.append((concat_cited, ans))

    # 2) each individual passage → entail?
    for cid in cited_ids:
        text = next(txt for did, txt, _ in hits if did == cid)
        pairs.append((text, ans))

    # 3) “others” without each passage → entail?
    for cid in cited_ids:
        others = [
            next(txt for did2, txt, _ in hits if did2 == oid)
            for oid in cited_ids if oid != cid
        ]
        if others:
            pairs.append(("\n\n".join(others), ans))

    # batch‐run all entailment checks
    entailments = batch_entails(pairs)

    #────────────────────────────────────────
    # unpack results
    #────────────────────────────────────────
    idx = 0
    cit_recall = 0.0
    cit_prec   = 0.0

    if cited_ids:
        # first slot is joint concat→entail?
        cit_recall = float(entailments[idx])
        idx += 1

        n = len(cited_ids)
        indiv_entails  = entailments[idx : idx + n]
        idx += n
        others_entails = entailments[idx : idx + n]
        idx += n

        # precision = fraction of cited passes that are necessary
        good = sum(
            1 for i in range(n)
            if indiv_entails[i] and not others_entails[i]
        )
        cit_prec = good / n

    #────────────────────────────────────────
    # retrieval‐recall@K
    #────────────────────────────────────────
    if cited_ids:
        ret_recall_k = len(set(retrieved_ids) & set(cited_ids)) / len(cited_ids)
    else:
        ret_recall_k = 0.0

    out_rows.append({
        "question":       q,
        "model_answer":   ans,
        "cited_ids":      cited_ids,
        "retrieved_ids":  retrieved_ids,
        "CitationRecall": cit_recall,
        "CitationPrec":   cit_prec,
        "RetrievalRec@K": ret_recall_k
    })

# ─────────────────────────────────────────────────────────────────────────────
# 5) assemble & save
# ─────────────────────────────────────────────────────────────────────────────
df = pd.DataFrame(out_rows)
print("Aggregates:\n", df[["CitationRecall","CitationPrec","RetrievalRec@K"]].mean())
df.to_csv(os.path.join(WORKSPACE_PATH, "eli5_alce_results.csv"), index=False)



Loaded 325475 examples


Scoring examples:   0%|          | 0/325475 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
import os
import re
import json
import pandas as pd
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ─────────────────────────────────────────────────────────────────────────────
# 1) Read your data
# ─────────────────────────────────────────────────────────────────────────────
WORKSPACE_PATH = "/data/horse/ws/uskh580e-llm_citation_ws"
ELI5_PATH      = os.path.join(WORKSPACE_PATH, "eli5_normed_slice1k.jsonl")
records        = [json.loads(line) for line in open(ELI5_PATH, "r")]
print(f"Loaded {len(records)} examples")

# ─────────────────────────────────────────────────────────────────────────────
# 2) Pick device & load model
# ─────────────────────────────────────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

nli_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(nli_name)

if device.type == "cuda":
    # load in float16 directly onto GPU
    model = AutoModelForSequenceClassification.from_pretrained(
        nli_name,
        torch_dtype=torch.float16,
        device_map="auto"             # Hf will place weights on your GPU
    )
else:
    # fallback to CPU float32
    model = AutoModelForSequenceClassification.from_pretrained(
        nli_name,
        low_cpu_mem_usage=True
    ).to(device)

model.eval()

# ─────────────────────────────────────────────────────────────────────────────
# 3) Batched entailment checker
# ─────────────────────────────────────────────────────────────────────────────
def batch_entails(pairs, batch_size=16):
    results = []
    for i in range(0, len(pairs), batch_size):
        chunk = pairs[i : i + batch_size]
        ps, hs = zip(*chunk)
        toks = tokenizer(
            list(ps),
            list(hs),
            return_tensors="pt",
            truncation=True,
            padding=True
        ).to(device)
        with torch.no_grad():
            logits = model(**toks).logits
        preds = torch.argmax(logits, dim=-1).cpu().tolist()
        results.extend([p == 2 for p in preds])
    return results

# ─────────────────────────────────────────────────────────────────────────────
# 4) Citation‐evaluation loop
# ─────────────────────────────────────────────────────────────────────────────
cite_re = re.compile(r"\[Corpus ID:\s*(\d+)\]")
RETRIEVE_K = 20

rows = []
for ex in tqdm(records, desc="Scoring", total=len(records)):
    q = ex["question"]
    ans = answer_with_citations_scads(q, strategy="Academic Aspiration (2-stage)")
    cited = [int(m) for m in cite_re.findall(ans)]
    hits  = retrieve_top_k(q, k=RETRIEVE_K)
    retrieved = [did for did, _, _ in hits]

    # build (premise,hypothesis) checks
    checks = []
    if cited:
        # 1) all cited passages concat → ans
        full = "\n\n".join(txt for did, txt, _ in hits if did in cited)
        checks.append((full, ans))
        # 2) each cited pass → ans
        for cid in cited:
            txt = next(txt for did, txt, _ in hits if did == cid)
            checks.append((txt, ans))
        # 3) everything except each → ans
        for cid in cited:
            others = [
                next(txt for did2, txt, _ in hits if did2 in cited and did2 != cid)
            ]
            if others:
                checks.append(("\n\n".join(others), ans))

    ent = batch_entails(checks)

    # unpack results
    idx = 0
    recall  = float(ent[0]) if cited else 0.0
    idx += 1
    prec = 0.0
    if cited:
        n = len(cited)
        indiv = ent[idx:idx+n]
        idx += n
        others = ent[idx:idx+n]
        # precision = fraction of cited that are *necessary*
        good = sum(1 for i in range(n) if indiv[i] and not others[i])
        prec = good / n
    # retrieval‐recall@K
    ret_r = len(set(retrieved) & set(cited)) / len(cited) if cited else 0.0

    rows.append({
        "question":        q,
        "answer":          ans,
        "cited_ids":       cited,
        "retrieved_ids":   retrieved,
        "CitationRecall":  recall,
        "CitationPrec":    prec,
        "RetrievalRec@K":  ret_r
    })

# ─────────────────────────────────────────────────────────────────────────────
# 5) Save & report
# ─────────────────────────────────────────────────────────────────────────────
df = pd.DataFrame(rows)
print("Averages:\n", df[["CitationRecall","CitationPrec","RetrievalRec@K"]].mean())
df.to_csv(os.path.join(WORKSPACE_PATH, "eli5_alce_results.csv"), index=False)


Loaded 325475 examples
Using device: cuda


Scoring:   0%|          | 0/325475 [00:00<?, ?it/s]

KeyboardInterrupt: 